<a href="https://colab.research.google.com/github/hiris25/TEVC-analysis-scripts/blob/master/DR_antagonists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TEVC - Dose Response - Antagonists**

## Set up libraries

In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns
#import scipy.optimize as opt
#from scipy import stats
import glob as glob
#import matplotlib.ticker as ticker

#%config InlineBackend.figure_format = 'retina'

## Import and prepare data

### Import Files

Import all files from specified folder and combine into one dataframe called 'df'

In [11]:
path = r'/Users/irishardege/Library/CloudStorage/OneDrive-UniversityofCambridge/Zoology/DATA/TEVC/acr21_lgc4/acr21_lgc4_antagonists' # use your path
all_files = glob.glob(path + "/*Datatable.dat") # use your extension

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep='\t') # for .dat files you must specify tab separation sep='\t'
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

### Remove excess coloumns and clean data
Here we are keeping only the relevant information, you can add or remove coloumns as you like.
We will also be standardising the units and generally cleaning up the data

In [35]:
data = df[['Well', 'Buffer', 'Comp. 1', 'conc. 1', 'unit 1', 'Minimum', 'Baseline Average', 'Start Date', 'Comp. 2', 'conc. 2', 'unit 2']].copy()
data['date'] = data['Start Date'].str.extract(r'(\d{2}/\d{2}/\d{4})', expand=True)
data['unique_id'] = data['Well'] + '_' + data['date']

Transform units all to uM scale

In [40]:
conditions = [
    (data['unit 1'] == 'mM'),
    (data['unit 1'] == 'µM'),
    (data['unit 1'] == 'nM')]
choices = ['1000', '1', '0.001']
data['factor'] = np.select(conditions, choices, default='0')

data['factor']=data.factor.astype(float)

data['Comp1_Concentration_µM'] = data['conc. 1'] * data['factor']

conditions = [
    (data['unit 2'] == 'mM'),
    (data['unit 2'] == 'µM'),
    (data['unit 2'] == 'nM')]
choices = ['1000', '1', '0.001']
data['factor'] = np.select(conditions, choices, default='0')

data['factor']=data.factor.astype(float)

data['Comp2_Concentration_µM'] = data['conc. 2'] * data['factor']

data_clean = data[['unique_id','Well', 'Buffer', 'Comp. 1', 'Comp. 2', 'Minimum', 'Comp2_Concentration_µM', 'date', 'Comp1_Concentration_µM']].copy()

#decide if comp1 or comp2 is agonist or antagonsit

#data['log_Agonist_µM'] = np.log(data['Comp1_Concentration_µM'])
#data['log_Antagonist_µM'] = np.log(data['Comp2_Concentration_µM'])

data_clean['Antagonist'] = data_clean['Comp. 2']
data_clean['Agonist'] = data_clean['Comp. 1']

data_clean['Response'] = data_clean['Minimum']
data_clean['Agonist_µM'] = data_clean['Comp1_Concentration_µM']
data_clean['Antagonist_µM'] = data_clean['Comp2_Concentration_µM']

data_clean = data_clean.drop(columns=['Comp. 1'])
data_clean = data_clean.drop(columns=['Comp. 2'])
data_clean = data_clean.drop(columns=['Comp1_Concentration_µM'])
data_clean = data_clean.drop(columns=['Comp2_Concentration_µM'])

data_clean.reset_index(drop=True)    

,unique_id,Well,Buffer,Minimum,date,Antagonist,Agonist,Response,Agonist_µM,Antagonist_µM
0,A1_24/02/2025,A1,ND96,-863,24/02/2025,empty,Dopamine,-863,30.0,0.0
1,A1_24/02/2025,A1,ND96,-841,24/02/2025,empty,Dopamine,-841,30.0,0.0
2,A1_24/02/2025,A1,ND96,-725,24/02/2025,empty,Dopamine,-725,30.0,0.0
3,A1_24/02/2025,A1,ND96,-737,24/02/2025,empty,Dopamine,-737,30.0,0.0
4,A1_24/02/2025,A1,ND96,-1201,24/02/2025,empty,Dopamine,-1201,30.0,0.0
...,...,...,...,...,...,...,...,...,...,...
103,B8_24/02/2025,B8,ND96,-5932,24/02/2025,Isoprenaline,Dopamine,-5932,30.0,0.5
104,B8_24/02/2025,B8,ND96,-4424,24/02/2025,Isoprenaline,Dopamine,-4424,30.0,5.0
105,B8_24/02/2025,B8,ND96,-3293,24/02/2025,Isoprenaline,Dopamine,-3293,30.0,50.0
106,B8_24/02/2025,B8,ND96,-2395,24/02/2025,Isoprenaline,Dopamine,-2395,30.0,500.0


## Transform data to I/Imax for each oocyte and each compound(optional)

comment out if not you do **NOT** want to normalise data

In [62]:
# Group by 'unique_id' and find the index of the first row where 'Antagonist' is not 'empty'
Agonist_control_index = data_clean.groupby('unique_id').apply(
    lambda group: group[group['Antagonist'] != 'empty'].index.min()-1
).reset_index(name='Agonist_control_index')

# Merge the DataFrames on 'unique_id' to align with 'Agonist_control_index'
data_clean_norm_special = pd.merge(Agonist_control_index, data_clean, on='unique_id')

# Use the 'Agonist_control_index' to get the corresponding 'Response' values
data_clean_norm_special['Response_at_Agonist_control'] = merged.apply(
    lambda row: data_clean.loc[row['Agonist_control_index'], 'Response'], axis=1
)

data_clean_norm_special['I/Imax'] = data_clean_norm_special['Response'] / data_clean_norm_special['Response_at_Agonist_control']

data_clean_norm_special.head()

,unique_id,Agonist_control_index,Well,Buffer,Minimum,date,Antagonist,Agonist,Response,Agonist_µM,Antagonist_µM,Response_at_Agonist_control,I/Imax
0,A1_24/02/2025,5,A1,ND96,-863,24/02/2025,empty,Dopamine,-863,30.0,0.0,-966,0.893375
1,A1_24/02/2025,5,A1,ND96,-841,24/02/2025,empty,Dopamine,-841,30.0,0.0,-966,0.870600
2,A1_24/02/2025,5,A1,ND96,-725,24/02/2025,empty,Dopamine,-725,30.0,0.0,-966,0.750518
3,A1_24/02/2025,5,A1,ND96,-737,24/02/2025,empty,Dopamine,-737,30.0,0.0,-966,0.762940
4,A1_24/02/2025,5,A1,ND96,-1201,24/02/2025,empty,Dopamine,-1201,30.0,0.0,-966,1.243271


In [59]:
# Drop rows where 'Antagonist' is 'empty' and 'I/Iamx' is not 1
data_clean_norm_special_final = data_clean_norm_special[~((data_clean_norm_special['Antagonist'] == 'empty') & (data_clean_norm_special['I/Imax'] != 1))]
data_clean_norm_special_final.head()

,unique_id,Agonist_control_index,Well,Buffer,Minimum,date,Antagonist,Agonist,Response,Agonist_µM,Antagonist_µM,Response_at_Agonist_control,I/Imax
5,A1_24/02/2025,5,A1,ND96,-966,24/02/2025,empty,Dopamine,-966,30.0,0.00,-966,1.000000
6,A1_24/02/2025,5,A1,ND96,-636,24/02/2025,Nicotine,Dopamine,-636,30.0,0.05,-966,0.658385
7,A1_24/02/2025,5,A1,ND96,-374,24/02/2025,Nicotine,Dopamine,-374,30.0,0.50,-966,0.387164
8,A1_24/02/2025,5,A1,ND96,-351,24/02/2025,Nicotine,Dopamine,-351,30.0,5.00,-966,0.363354
9,A1_24/02/2025,5,A1,ND96,-363,24/02/2025,Nicotine,Dopamine,-363,30.0,50.00,-966,0.375776


In [63]:
#normalisation with the min response rather than specific well as above
minvalues = data_clean.groupby(['unique_id'])["Minimum"].min()
norm = data_clean.Minimum / data_clean.groupby(['unique_id']).Minimum.transform(np.min)
data_clean_normalised = data_clean.copy()
data_clean_normalised['I/Imax'] = norm

data_clean_normalised.head()

,unique_id,Well,Buffer,Minimum,date,Antagonist,Agonist,Response,Agonist_µM,Antagonist_µM,I/Imax
0,A1_24/02/2025,A1,ND96,-863,24/02/2025,empty,Dopamine,-863,30.0,0.0,0.718568
1,A1_24/02/2025,A1,ND96,-841,24/02/2025,empty,Dopamine,-841,30.0,0.0,0.700250
2,A1_24/02/2025,A1,ND96,-725,24/02/2025,empty,Dopamine,-725,30.0,0.0,0.603664
3,A1_24/02/2025,A1,ND96,-737,24/02/2025,empty,Dopamine,-737,30.0,0.0,0.613655
4,A1_24/02/2025,A1,ND96,-1201,24/02/2025,empty,Dopamine,-1201,30.0,0.0,1.000000


## Exclude outliers and whole oocytes

### Exclude specific oocytes (optional)
Use the code below to exclude data for indivdiual oocytes (rows)

In [ ]:
#data = data[~data['Well'].isin(['A1'])]

(84, 12)

## Calculate means and export data

### Export as .csv
comment out if you do not want to export .csv
change path and file name before running

exports grouped mean without outliers, std, N into a new .csv file

Includes Agonist and log agonist concentrations in uM

In [65]:
export = data_clean_norm_special_final.groupby(['Antagonist', 'Antagonist_µM'])

export = export['I/Imax'].describe()

export.head(15)

count      mean       std       min       25%  \
Antagonist   Antagonist_µM                                                  
Isoprenaline 0.05             6.0  0.775198  0.161382  0.577619  0.676341   
             0.50             6.0  0.547178  0.215475  0.310477  0.391811   
             5.00             6.0  0.384499  0.158072  0.228195  0.273016   
             50.00            6.0  0.303099  0.103285  0.209545  0.223662   
             500.00           6.0  0.278012  0.049393  0.225562  0.242153   
Nicotine     0.05             7.0  0.790828  0.087762  0.658385  0.729884   
             0.50             7.0  0.570946  0.149175  0.384775  0.434559   
             5.00             7.0  0.439094  0.137370  0.270844  0.330831   
             50.00            7.0  0.378656  0.120210  0.242361  0.314003   
             500.00           7.0  0.203269  0.071217  0.137752  0.149669   
empty        0.00            13.0  1.000000  0.000000  1.000000  1.000000   

                                 50%       75%       max  
Antagonist   Antagonist_µM                                
Isoprenaline 0.05           0.743942  0.876007  1.011847  
             0.50           0.515617  0.671290  0.867632  
             5.00           0.343721  0.457063  0.647067  
             50.00          0.281822  0.340069  0.481644  
             500.00         0.265272  0.311733  0.350300  
Nicotine     0.05           0.810499  0.855272  0.896601  
             0.50           0.656290  0.672086  0.742268  
             5.00           0.478102  0.499910  0.663230  
             50.00          0.374410  0.398782  0.608247  
             500.00         0.177329  0.238425  0.331615  
empty        0.00           1.000000  1.000000  1.000000

In [ ]:
export.to_csv(r'/Users/irishardege/Library/CloudStorage/OneDrive-UniversityofCambridge/Zoology/DATA/TEVC/acr21_lgc4/acr21_lgc4_antagonists/isoprenaline_nicotine_DRs_means.csv')

#data.to_csv(r'/Users/irishardege/Library/CloudStorage/OneDrive-UniversityofCambridge/Zoology/DATA/TEVC/acr21_lgc4/acr21_lgc4_antagonists/isoprenaline_nicotine_DRs.csv')